# Yelp API - Lab


## Introduction 

Now that we've seen how the Yelp API works and some basic Folium visualizations, it's time to put those skills to work in order to create a working map! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to: 
* Create HTTP requests to get data from Yelp API
* Parse HTTP responses and perform data analysis on the data returned
* Perform pagination to retrieve troves of data!
* Create a simple geographical system on to view information about selected businesses, at a given location. 

## Problem Introduction

You've now worked with some API calls, but we have yet to see how to retrieve a more complete dataset in a programmatic manner. Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retrieving 50 at a time. Processes such as these are often referred to as pagination.

In this lab, you will define a search and then paginate over the results to retrieve all of the results. You'll then parse these responses as a DataFrame (for further exploration) and create a map using Folium to visualize the results geographically.

## Part I - Make the Initial Request

Start by making an initial request to the Yelp API. Your search must include at least 2 parameters: **term** and **location**. For example, you might search for pizza restaurants in NYC. The term and location is up to you, but make the request below.

In [62]:
#Your code here
import requests
import json
import pandas as pd
import folium
import numpy as np

def yelp_request (offset, limit):
    client_id = 'anF6Qsej_5ZGg0iFwWkKWA'
    api_key = 'sG8lXeonnc0lFO2zUVtJY6fWKoqTUEcTTWGajtdDysQsPSKKAsQxDO27_KzTStavl6g88t4PWV9WxwFV1TezThRQMb_hU1fDBOWG2wEYaF03icHPjF8DV4QCwSrcXHYx'

    term = 'Georgian'
    location = 'New York NY'
    SEARCH_LIMIT = limit
    offset = offset

    url = 'https://api.yelp.com/v3/businesses/search'

    headers = {
            'Authorization': 'Bearer {}'.format(api_key),
        }

    url_params = {
                    'term': term.replace(' ', '+'),
                    'location': location.replace(' ', '+'),
                    'limit': SEARCH_LIMIT,
                    'offset' : offset
                }
    try:
        response = requests.get(url, headers=headers, params=url_params)
    except Exception as e:
        print(e)
    finally:
        pass
    
    return response

offset = 0
step = 5
response = yelp_request(offset, interval)
print(response)
print(type(response.text))
print(response.text[:1000])
data = response.json()
data.keys()
total_count = data['total']


<Response [200]>
<class 'str'>
{"businesses": [{"id": "MbwsCB7echBlVwzfJ_9c9g", "alias": "old-tbilisi-garden-new-york-5", "name": "Old Tbilisi Garden", "image_url": "https://s3-media4.fl.yelpcdn.com/bphoto/rHIAAjkXEWpF1_HlHR61Mw/o.jpg", "is_closed": false, "url": "https://www.yelp.com/biz/old-tbilisi-garden-new-york-5?adjust_creative=anF6Qsej_5ZGg0iFwWkKWA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=anF6Qsej_5ZGg0iFwWkKWA", "review_count": 345, "categories": [{"alias": "modern_european", "title": "Modern European"}, {"alias": "wine_bars", "title": "Wine Bars"}, {"alias": "georgian", "title": "Georgian"}], "rating": 4.0, "coordinates": {"latitude": 40.7287707, "longitude": -74.0008161}, "transactions": ["delivery", "restaurant_reservation", "pickup"], "price": "$$", "location": {"address1": "174 Bleecker St", "address2": "", "address3": "", "city": "New York", "zip_code": "10012", "country": "US", "state": "NY", "display_address": ["174 Bleecker St", "New York,

In [63]:
total_count

56

## Pagination

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. Your final goal will be to reformat the data concerning the businesses themselves into a pandas DataFrame from the json objects.

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and are also can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

In [64]:
# Your code here; use a function or loop to retrieve all the results from your original request

businesses = []
while offset < (total_count + step):
    try:
        response = yelp_request(offset, step)
        data = response.json()
        for business in data['businesses']:
            businesses.append(business)
    except Exception as e:
        print(e)
    
    finally:
        offset += step  

df = pd.DataFrame.from_dict(businesses)
df.head()

,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,old-tbilisi-garden-new-york-5,"[{'alias': 'modern_european', 'title': 'Modern...","{'latitude': 40.7287707, 'longitude': -74.0008...",(212) 470-6064,2651.331111,MbwsCB7echBlVwzfJ_9c9g,https://s3-media4.fl.yelpcdn.com/bphoto/rHIAAj...,False,"{'address1': '174 Bleecker St', 'address2': ''...",Old Tbilisi Garden,+12124706064,$$,4.0,345,"[restaurant_reservation, pickup, delivery]",https://www.yelp.com/biz/old-tbilisi-garden-ne...
1,chama-mama-new-york,"[{'alias': 'georgian', 'title': 'Georgian'}]","{'latitude': 40.7384, 'longitude': -73.99892}",(646) 438-9007,3690.546028,cTE0e96GYzet799EJeStoQ,https://s3-media3.fl.yelpcdn.com/bphoto/qRZaBn...,False,"{'address1': '149 W 14th St', 'address2': None...",Chama Mama,+16464389007,NaN,4.5,58,[],https://www.yelp.com/biz/chama-mama-new-york?a...
2,oda-house-new-york,"[{'alias': 'georgian', 'title': 'Georgian'}]","{'latitude': 40.7240282, 'longitude': -73.9821...",(212) 353-3838,2304.651268,dYqWosnE1UlJs4Wx-M_u_A,https://s3-media1.fl.yelpcdn.com/bphoto/0zLOP8...,False,"{'address1': '76 Ave B', 'address2': '', 'addr...",Oda House,+12123533838,$$,4.0,328,"[pickup, delivery]",https://www.yelp.com/biz/oda-house-new-york?ad...
3,marani-rego-park-5,"[{'alias': 'mediterranean', 'title': 'Mediterr...","{'latitude': 40.731202, 'longitude': -73.859833}",(718) 569-0600,11688.287889,A0GVSLdYCEIr-yLZnqhJkg,https://s3-media4.fl.yelpcdn.com/bphoto/Vlzy84...,False,"{'address1': '97-26 63rd Rd', 'address2': '', ...",Marani,+17185690600,$$,4.5,221,"[pickup, delivery]",https://www.yelp.com/biz/marani-rego-park-5?ad...
4,cheeseboat-brooklyn,"[{'alias': 'georgian', 'title': 'Georgian'}]","{'latitude': 40.71994, 'longitude': -73.95791}",(929) 295-8401,3462.122520,Z3yjBSmuoQbz6olnO_S1tQ,https://s3-media4.fl.yelpcdn.com/bphoto/MK_u1l...,False,"{'address1': '80 Berry St', 'address2': None, ...",Cheeseboat,+19292958401,$$,4.0,351,"[pickup, delivery]",https://www.yelp.com/biz/cheeseboat-brooklyn?a...


In [65]:
df.tail()
df.shape
df.columns

Index(['alias', 'categories', 'coordinates', 'display_phone', 'distance', 'id',
       'image_url', 'is_closed', 'location', 'name', 'phone', 'price',
       'rating', 'review_count', 'transactions', 'url'],
      dtype='object')

## Exploratory Analysis

Take the restaurants from the previous question and do an initial exploratory analysis. At minimum, this should include looking at the distribution of features such as price, rating and number of reviews as well as the relations between these dimensions.

In [67]:
df1 = df.loc[:,['coordinates', 'is_closed', 'location', 'name', 'phone', 'price', 'rating', 'review_count']]
df1.columns
             
             

Index(['coordinates', 'is_closed', 'location', 'name', 'phone', 'price',
       'rating', 'review_count'],
      dtype='object')

In [73]:
df1.head()
data['region']

{'center': {'longitude': -73.99429321289062, 'latitude': 40.70544486444615}}

## Mapping

Look at the initial Yelp example and try and make a map using Folium of the restaurants you retrieved. Be sure to also add popups to the markers giving some basic information such as name, rating and price.

In [85]:
#Your code here
start_lat = data['region']['center']['latitude']
start_long = data['region']['center']['longitude']
base_map = folium.Map([start_lat, start_long], zoom_start=13)

for b in businesses:
    lat = b['coordinates']['latitude']
    long = b['coordinates']['longitude']
    popup_text = "{}, {}, {} {}".format(b['location']['address1'], b['location']['city'], b['location']['state'], b['location']['zip_code'])
    popup = folium.Popup(popup_text, parse_html=True)
    marker = folium.Marker(location=[lat, long], popup=popup)
    marker.add_to(base_map)
base_map



## Summary

Nice work! In this lab, you synthesized your skills for the day, making multiple API calls to Yelp in order to paginate through a results set, performing some basic exploratory analysis and then creating a nice map visual to display the results! Well done!